In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [38]:
import os, glob
import duckdb
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, fixed

# Brukerhistorie 1, Kjør funksjonene nedenfor

## Sett inn verdier og kjør funskjonen

In [41]:


def get_filtered_ais_data_spatial(
    base_path: str,
    ship_type: int,
    center_lat: float,
    center_lon: float,
    buffer_radius_m: float,
    start_time: str,
    end_time: str,
    max_rows: int
) -> pd.DataFrame:
    conditions_dict = {"ship_type": ship_type}

    parquet_pattern = os.path.join(base_path, "**", "*.parquet*")
    files = glob.glob(parquet_pattern, recursive=True)
    if not files:
        raise IOError(f"Ingen filer funnet for mønster {parquet_pattern!r}")

    con = duckdb.connect()
    con.execute("INSTALL spatial;")
    con.execute("LOAD spatial;")

    query = f"SELECT * FROM read_parquet('{parquet_pattern}')"
    where = []

    for col, val in conditions_dict.items():
        if val:
            where.append(f"{col} = {val}")

    if start_time:
        where.append(f"date_time_utc >= TIMESTAMP '{start_time}'")
    if end_time:
        where.append(f"date_time_utc <= TIMESTAMP '{end_time}'")

    radius_deg = buffer_radius_m / 111000.0
    where.append(
        f"ST_Distance(geometry, ST_GeomFromText('POINT({center_lon} {center_lat})')) <= {radius_deg}"
    )

    if where:
        query += " WHERE " + " AND ".join(where)
    query += " ORDER BY date_time_utc"
    if max_rows:
        query += f" LIMIT {max_rows}"

    df = con.execute(query).fetchdf()
    con.close()
    return df

def render_ais_map(
    df: pd.DataFrame,
    center_lat: float,
    center_lon: float,
    buffer_radius_m: float
) -> folium.Map:
    m = folium.Map(location=[center_lat, center_lon],
                   zoom_start=12,
                   tiles="cartodb positron")
    folium.Circle(
        location=[center_lat, center_lon],
        radius=buffer_radius_m,
        color='blue',
        fill=True,
        fill_opacity=0.1,
        popup="Bufferområde"
    ).add_to(m)

    cluster = MarkerCluster().add_to(m)
    for _, row in df.iterrows():
        if pd.notnull(row.longitude) and pd.notnull(row.latitude):
            folium.Marker(
                location=[row.latitude, row.longitude],
                popup=(
                    f"MMSI: {row.mmsi}<br>"
                    f"Time: {row.date_time_utc}<br>"
                    f"Lon/Lat: {row.longitude:.4f}, {row.latitude:.4f}"
                ),
                icon=folium.Icon(color='red', icon='info-sign')
            ).add_to(cluster)
    return m

# GUI-komponenter med ipywidgets
def interactive_ais_query():
    interact(run_query,
        base_path=widgets.Text(value='./data/processed', description='Base Path:'),
        ship_type=widgets.IntText(value=51, description='Ship Type:'),
        center_lat=widgets.FloatText(value=58.142359, description='Center Lat:'),
        center_lon=widgets.FloatText(value=8.025218, description='Center Lon:'),
        buffer_radius_m=widgets.IntText(value=6000, description='Radius (m):'),
        start_time=widgets.Text(value='2025-01-21 13:00:00', description='Start Time:'),
        end_time=widgets.Text(value='2025-01-21 14:00:00', description='End Time:'),
        max_rows=widgets.IntText(value=10, description='Max Rows:')
    )

  # Ferdig-knapp
done_button = widgets.Button(
    description="Done",
    button_style='info',
    layout=widgets.Layout(width='150px')
)


def run_query(base_path, ship_type, center_lat, center_lon, buffer_radius_m, start_time, end_time, max_rows):
    df = get_filtered_ais_data_spatial(
        base_path,
        ship_type,
        center_lat, center_lon,
        buffer_radius_m,
        start_time,
        end_time,
        max_rows
    )

    print("Rader hentet:", len(df))
    display(df.head()) #Ta bort head for å vise hele tabellen

    m = render_ais_map(df, center_lat, center_lon, buffer_radius_m)
    display(m)

# Kjør GUI
interactive_ais_query()

interactive(children=(Text(value='./data/processed', description='Base Path:'), IntText(value=51, description=…